In [1]:
import boto3
import fitz
import os

from botocore.config import Config
from dotenv import load_dotenv
from langchain_aws import ChatBedrock
from langchain_core.output_parsers import StrOutputParser
from langchain.prompts import PromptTemplate

load_dotenv()

aws_default_region = os.getenv('AWS_DEFAULT_REGION')
aws_secret_access_key = os.getenv('AWS_SECRET_ACCESS_KEY')
aws_access_key_id = os.getenv('AWS_ACCESS_KEY_ID')

def get_model():
    # configuration for retries
    retry_config = Config(
        region_name='us-east-1',
        retries={
            'max_attempts': 10,
            'mode': 'standard'
        }
    )
    # initialise Bedrock runtime client
    bedrock_runtime = boto3.client("bedrock-runtime", config=retry_config)

    # model kwargs
    model_id = "anthropic.claude-3-5-sonnet-20240620-v1:0" # adjust as needed
    # model_id ="anthropic-beta: max-tokens-3-5-sonnet-2024-07-15"
    model_kwargs = {
        "max_tokens": 8192,
        "temperature": 0.0,
        "top_k": 250,
        "top_p": 1,
        "stop_sequences": ["\n\nHuman"],
    }
    
    llm = ChatBedrock(
        client=bedrock_runtime,
        model_id=model_id,
        model_kwargs=model_kwargs,
    )
    return llm

llm = get_model()

In [2]:
def remove_after_references(text):
    # Define the substring to search for
    marker = "References"
    
    # Find the index of the substring
    index = text.find(marker)
    
    # If the substring is found, slice the string up to the index
    if index != -1:
        return text[:index + len(marker)]
    
    # If the substring is not found, return the original text
    return text

def extract_pdf_text(pdf_path):
    # Open the PDF
    doc = fitz.open(pdf_path)
    text = ""
    
    # Iterate through the pages and extract text
    for page in doc:
        text += page.get_text()
    return remove_after_references(text)

In [3]:
PDF_PATH = "pdfs/Project MUSE - Hard Choices.pdf"
PDF_TEXT = extract_pdf_text(PDF_PATH)

In [4]:
answer_prompt = """<system>
You are an expert academic writer tasked with critically reflecting on PDF articles. Focus on providing analytical depth, objective assessment, and identifying key points.
</system>

<instructions>
You are given a PDF article for analysis. Please analyze the article and write a 300-word critical reflection on the following topic: {topic}.

Ensure that the reflection does not focus on personal opinions but critically evaluates the article's arguments and approach. Your reflection should include:

1. An appraisal of how well the author substantiates their argument.
2. A discussion of any new directions or insights introduced by the article.
3. An evaluation of the methods used by the author, identifying whether they support or undermine the argument.
4. Any interesting or thought-provoking points raised.
5. Connections between the article and broader themes or other readings relevant to the topic.

Focus on maintaining an objective tone and a critical approach to assessing the article's strengths and limitations. Do not include any preambles in your reflection.
</instructions>

<pdf_text>
{pdf_text}
</pdf_text>
"""

answer_prompt_template = PromptTemplate(
    template=answer_prompt,
    input_variables=["topic", "pdf_text"]
)

In [6]:
answer_chain = answer_prompt_template | llm | StrOutputParser()

In [7]:
TOPIC = "Economy"
response = answer_chain.invoke({"topic": TOPIC, "pdf_text": PDF_TEXT})

In [8]:
print(response)

This critical reflection analyzes the chapter "The Four Myths of Inequality in Singapore" by Donald Low, focusing on the first myth discussed in the text.

The author effectively challenges the commonly held belief that inequality is a necessary counterpart of economic dynamism and competitiveness. Low presents this myth as a widely accepted assumption in Singapore's policy circles and among the general population. He argues that this belief has attained an almost mythical status, forming a paradigm against which alternative ideas for social security are evaluated.

The chapter introduces new insights by questioning the validity of this assumption in light of economic theory and evidence. Low suggests that policymakers should approach the problem of inequality pragmatically, free from dogma and ideology. This approach represents a significant departure from the prevailing mindset in Singapore's governance.

The author substantiates his argument by referencing international evidence and